# Implementing YOLO v1 in PyTorch

This notebook implements...

----
## Data Sources
----

For our training data input, we resize each image to 112px by 112px and apply a greyscale filter. We take the pixel values in a 112x112 matrix and scale the values between -1 and 1. We take all of these scaled image matrices and feed them into our dataloader.

<div>
     <center><img src="./img/imageinput.png" width="750"/> </center>
</div>

----
## Importing Data
----

blah blah blah...

----
## Model Architecture
----

blah blah blah...

----
## Loss Function
----

YOLO v1 implements a custom loss function, described in their paper in the following equation:

<div>
     <center><img src="./img/lossfunction.png" width="750"/> </center>
</div>

This may look overwhelming at first, but it's not overly complicated. The loss function takes the SSE across the predicted $(x,y)$ box midpoint, $(h,w)$ box dimensions, and the set of $c_i$ category probabilities. Some other symbols are:

$𝟙_{i}^{\textrm{obj}}$ indicates whether an object appears in cell $i$

$𝟙_{ij}^{\textrm{obj}}$ indicates whether the $j^{\textrm{th}}$ bounding box predictor in cell $i$ is "responsible" for that prediction (highest IOU)

$\lambda_{\textrm{coord}}$ is a learning parameter set to 5

$\lambda_{\textrm{noobj}}$ is a learning parameter set to 0.5

To implement this multi-part loss function, we first have to define a custom loss function in torch. We create a new `torch.nn.Module` for our custom loss, which we will call `YOLOLoss`:

In [1]:
import torch
import torch.nn

class YOLOLoss(torch.nn.Module):
    """
    Computes loss function according to YOLO v1.

    """
    def __init__(self, S = 7, B = 2, C = 20, l_coord = 5, l_noobj = 0.5):
        super().__init__()
        self.S = S
        self.B = B
        self.C = C
        self.l_coord = l_coord
        self.l_noobj = l_noobj
        self.mse = torch.nn.MSELoss(reduction="sum") # SSE instead of MSE

We then define a `forward` function for computing the loss. The full loss function, including the extensive `forward` function, can be found in `loss.py`. We borrow most of this code from https://github.com/aladdinpersson, as while it is not a conceptually difficult loss function, the actual code implementation with torch functions becomes a bit unweildy.